<a href="https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/309_Training_your_own_Chatbot_using_GPT%E2%80%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://youtu.be/DxygPxcfW_I

In [2]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 49.8 MB/s eta 0:00:00


In [4]:
!pip uninstall accelerate
!pip install accelerate==0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 4.6 MB/s eta 0:00:00


In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.7 MB/s eta 0:00:00


# **This code is an implementation of a chatbot using the GPT-2 language model from Hugging Face's Transformers library.**
<br>
Reference: https://huggingface.co/transformers/v2.2.0/pretrained_models.html



In [4]:
!pip install transformers
!pip install torch
!pip install -U PyPDF2
!pip install python-docx



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 4.9 MB/s eta 0:00:00


In [5]:
import os
import re
import PyPDF2
from PyPDF2 import PdfReader
import docx
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments


Required functions to read text from various files located in a directory. Files can be a mix of pdf, docx, or txt.

In [6]:

# Functions to read different file types
import PyPDF2

def read_pdf(file_path):
    pdfFile = open(file_path, 'rb')
    pdfReader = PyPDF2.PdfReader(pdfFile)

    text = ""
    for page in range(len(pdfReader.pages)):
        pageObj = pdfReader.pages[page]
        text += pageObj.extract_text()
    return text


def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text



The train_chatbot function uses the combined text data to train a GPT-2 model using the provided training arguments. The resulting trained model and tokenizer are then saved to a specified output directory.

In [7]:
def train_chatbot(directory, model_output_path, train_fraction=0.8):
    # Read documents from the directory
    combined_text = read_documents_from_directory(directory)
    combined_text = re.sub(r'\n+', '\n', combined_text).strip()  # Remove excess newline characters

    # Split the text into training and validation sets
    split_index = int(train_fraction * len(combined_text))
    train_text = combined_text[:split_index]
    val_text = combined_text[split_index:]

    # Save the training and validation data as text files
    # Assuming 'train_text' and 'val_text' contain your text data

    with open("train.txt", "w", encoding="utf-8", errors="replace") as f:
        f.write(train_text)
    with open("val.txt", "w", encoding="utf-8", errors="replace") as f:
        f.write(val_text)



    # Set up the tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl

    # Prepare the dataset
    train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
    val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Set up the training arguments
    training_args = TrainingArguments(
        output_dir=model_output_path,
        overwrite_output_dir=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=50,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir='./logs',
    )

    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()
    trainer.save_model(model_output_path)

    # Save the tokenizer
    tokenizer.save_pretrained(model_output_path)


The generate_response function takes a trained model, tokenizer, and a prompt string as input and generates a response using the GPT-2 model.

In [8]:
def generate_response(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


The main function is the entry point for the program. It specifies the path to the directory containing the training data and the path to the output directory for the trained model and tokenizer. It then trains the chatbot using the train_chatbot function and generates a response to a specified prompt using the generate_response function.

## Now, let us test the model.
<p>
Use the following code if you are only performing inference (generating text). This can be placed in a separate notebook.

In [9]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In the case of the GPT-2 tokenizer, the model uses a byte-pair encoding (BPE) algorithm, which tokenizes text into subword units. As a result, one word might be represented by multiple tokens.

For example, if you set max_length to 50, the generated response will be limited to 50 tokens, which could be fewer than 50 words, depending on the text.

In [10]:
def chat_with_bot(model, tokenizer, chat_history=None):
    while True:
        user_input = input("You: ")  # Get user input
        chat_history.append(user_input)  # Add user input to chat history

        # Generate a response using the chat history as context
        response = generate_response(model, tokenizer, chat_history)

        # Remove user's input from the bot's response
        response_without_input = response[len(user_input):].strip()

        print("Bot:", response_without_input)

def generate_response(model, tokenizer, chat_history, max_length=100):
    input_text = "\n".join(chat_history)  # Combine chat history into a single text

    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

def main():
    directory = "/content/data"  # Replace with the path to your directory containing the files
    model_output_path = "/content/drive/MyDrive/GPT-Model2.0/"

    # Train the chatbot (if not already trained)
    if not os.path.exists(model_output_path):
        train_chatbot(directory, model_output_path)

    # Load the fine-tuned model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_output_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

    chat_history = []  # Initialize an empty chat history

    # Start the conversation
    print("Bot: Hi there! How can I help you today?")
    chat_with_bot(model, tokenizer, chat_history)

if __name__ == "__main__":
    main()

Bot: Hi there! How can I help you today?
You: who is a human ressources manager ?
Bot: Human resource manager is a top-level manager responsible for the administration of all human resource activities and policies. He or She supervise employees' compensation, benefits, staffing, affirmative action, employee relations, health and safety, and training and development functions. They also oversee below mentioned professional human resources staff.
Employment & Labour Laws in India
8 Rights for Private Employees by the Indian Labour Laws
Workmen's Compensation Act, 1923
Employee
You: hi
Bot: o is a human ressources manager?
hi
every organisation wants right people with them but they come at a price. compensation? cant say with certainty but its a fact that most HR managers dont get paid very much for their services. its because of high cost of doing business in the industry and its also because of the high level of expertise required by the managers from which they come.
employee rewards? 

KeyboardInterrupt: ignored

In [12]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [22]:
%%writefile app.py
import streamlit as st
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import streamlit as st
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load your fine-tuned chatbot model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/GPT-Modelv2")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/GPT-Modelv2")

# Add CSS styling
st.write(
    """
    <style>
    .container {
        display: flex;
        flex-direction: column;
        align-items: center;
    }
    .chat-container {
        border: 1px solid #ddd;
        padding: 20px;
        border-radius: 10px;
        background-color: #f5f5f5;
        max-width: 600px;
    }
    .user-message {
        background-color: #007BFF;
        color: white;
        padding: 10px;
        border-radius: 5px;
        max-width: 80%;
        text-align: right;
        margin: 5px 0;
    }
    .bot-message {
        background-color: #f0f0f0;
        color: #333;
        padding: 10px;
        border-radius: 5px;
        max-width: 80%;
        text-align: left;
        margin: 5px 0;
    }
    </style>
    """
)

# Set Streamlit app title and page config
st.title("Chatbot App")
st.set_page_config(page_title="Chatbot", page_icon="💬", layout="centered")

# Define chat history as a global variable
chat_history = []

# Create a centered container for chat
chat_container = st.container()

# Add a header and user input box
st.header("Chat with the Bot")
user_input = st.text_input("You:")

# Add a send button
if st.button("Send"):
    if user_input:
        response = chat_with_bot(user_input, chat_history)
        chat_history.append(f"You: {user_input}")
        chat_history.append(f"Bot: {response}")

# Display the chat history
for message in chat_history:
    if message.startswith("You:"):
        chat_container.markdown(f'<div class="user-message">{message}</div>', unsafe_allow_html=True)
    elif message.startswith("Bot:"):
        chat_container.markdown(f'<div class="bot-message">{message}</div>', unsafe_allow_html=True)

# Add a clear chat button
if st.button("Clear Chat"):
    chat_history = []

if __name__ == "__main__":
    st.write("Bot: Hi there! How can I help you today?")


Writing app.py


In [23]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.198.241.178:8501

  Stopping...
^C


In [25]:
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=019bc98f079ea6d2b888b4bae3742a66c389d23190472c5b859c0b01328525ed
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [29]:
!streamlit run app.py &>/content/logs.txt &

In [31]:
!curl ipv4.icanhazip.com

104.198.241.178


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.969s
your url is: https://rich-lights-serve.loca.lt
